## 11 - How to visualise data with xcube viewer in your workspace
### A DeepESDL example notebook 

This notebook demonstrates how to use the features of the xcube JupyterLab integration.
The notebook demonstrates three scenarios how xcube Viewer is utilized in JupyterLab.
In particular, we open xcube Viewer for any `xarray.Dataset` instances 

1. persisted in team s3 storage (saved datasets)
2. opened or otherwise created in this Notebook (in-memory datasets)
3. using a configuration file for customising styles. 

To explore the xcube viewer functionalities, please checkout the documentation: https://xcube.readthedocs.io/en/latest/viewer.html#functionality

Please, also refer to the [DeepESDL documentation](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/) and visit the platform's [website](https://www.earthsystemdatalab.net/) for further information!

Brockmann Consult, 2024

-----------------

**This notebook runs with the python environment `deepesdl-xcube-1.4.1`, please checkout the documentation for [help on changing the environment](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kerne).**

First, lets create a small cube, which we can visualise with xcube viewer. We will use ESA CCI data for this. Please head over to 03 - Generate CCI data cubes to get more details about the xcube-cci data store :)

In [1]:
from xcube.core.store import new_data_store
import os
import datetime

In [ ]:
store = new_data_store('ccizarr')

We request a dataset from the datasore:

In [3]:
def open_zarrstore(filename,time_range,variables):
    ds = store.open_data(filename)
    subset = ds.sel(time=slice(time_range[0], time_range[1]))
    subset = subset[variables]
    
    return subset

In [4]:
dataset = open_zarrstore('ESACCI-L4_GHRSST-SST-GMPE-GLOB_CDR2.0-1981-2016-v02.0-fv01.0.zarr', 
                         time_range=[datetime.datetime(1982,9,1),datetime.datetime(1982,9,7)],
                         variables=['analysed_sst'])

In [5]:
dataset

<xarray.Dataset> Size: 50MB
Dimensions:       (time: 6, lat: 720, lon: 1440)
Coordinates:
  * lat           (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) datetime64[ns] 48B 1982-09-01T12:00:00 ... 1982-09-0...
Data variables:
    analysed_sst  (time, lat, lon) float64 50MB dask.array<chunksize=(6, 720, 720), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.4
    acknowledgment:             Funded by ESA
    cdm_data_type:              grid
    comment:                    
    creator_email:              science.leader@esa-sst-cci.org
    creator_name:               SST_cci
    ...                         ...
    summary:                    An ensemble product with input from a number ...
    time_coverage_end:          20170101T000000Z
    time_coverage_start:        20161231T000000Z
    title:                      Global SST Ensemble, L4 GMPE
    uuid:                       dc0c5b25-93bf-4943-aba1-7f0de9109620
    westernmost_longitude:      -180.0

Next, save it to the team s3 storage:

To store the cube in your teams user space, please first retrieve the details from your environment variables as the following:

In [6]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

You need to instantiate a s3 datastore pointing to the team bucket:

In [7]:
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))


If you have stored no data to your user space, the returned list will be empty:

In [8]:
list(team_store.get_data_ids())

[]

In [9]:
output_id = 'analysed_sst.zarr'

The writing will take a few moments, as the data is global and will be persisted into the team s3 storage. 

In [10]:
team_store.write_data(dataset, output_id, replace=True)

'analysed_sst.zarr'

If you list the content of you datastore again, you will now see the newly written dataset in the list:

In [11]:
list(team_store.get_data_ids())

['analysed_sst.zarr']

Once the cube is stored in our team s3 storage, we can use xcube viewer jupyterlab extention to visualise it.

In [12]:
from xcube.webapi.viewer import Viewer

We use the xcube datastore framework here to open the dataset, but it could also be opened by other means, e.g., `xr.open_dataset()`, provided it has variables with dimensions ["time", "y", "x"] or ["y", "x"]. 

In [13]:
dataset = team_store.open_data(output_id)

In [14]:
dataset

<xarray.Dataset> Size: 50MB
Dimensions:       (time: 6, lat: 720, lon: 1440)
Coordinates:
  * lat           (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) datetime64[ns] 48B 1982-09-01T12:00:00 ... 1982-09-0...
Data variables:
    analysed_sst  (time, lat, lon) float64 50MB dask.array<chunksize=(6, 720, 720), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.4
    acknowledgment:             Funded by ESA
    cdm_data_type:              grid
    comment:                    
    creator_email:              science.leader@esa-sst-cci.org
    creator_name:               SST_cci
    ...                         ...
    summary:                    An ensemble product with input from a number ...
    time_coverage_end:          20170101T000000Z
    time_coverage_start:        20161231T000000Z
    title:                      Global SST Ensemble, L4 GMPE
    uuid:                       dc0c5b25-93bf-4943-aba1-7f0de9109620
    westernmost_longitude:      -180.0

---
**Scenario 1**: Open xcube Viewer for a dataset instances persisted from a certain source (saved datasets).

**For the functionalities of xcube viewer please head over to the documentation:** https://xcube.readthedocs.io/en/latest/viewer.html#functionality

In [ ]:
viewer = Viewer()

In [16]:
viewer.add_dataset(dataset)

'753844b4-220b-4c45-a23b-57231dcb7da3'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [17]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000
Viewer: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000


You can also open xcube Viewer inlined here:

In [18]:
viewer.show()

To stop the server and viewer:

In [19]:
viewer.stop_server()

---
**Scenario 2**: Open xcube Viewer for a dataset instances opened or otherwise created in this Notebook (in-memory datasets).

Below, let's fetch a dataset from the CCI zarr store, without persisting it. Be aware of performance loss, so in case you plan to use a dataset a lot for analysis or visualisation, please persist it into the team s3 storage. The public cubes provided within DeepESDL are already persisted in S3, so you should not duplicate them in your team storage. 

In [20]:
cci_dataset = open_zarrstore('ESACCI-L4_GHRSST-SST-GMPE-GLOB_CDR2.0-1981-2016-v02.0-fv01.0.zarr', 
                         time_range=[datetime.datetime(2016,9,1),datetime.datetime(2016,9,20)],
                         variables=['analysed_sst'])

In [21]:
cci_dataset

<xarray.Dataset> Size: 158MB
Dimensions:       (time: 19, lat: 720, lon: 1440)
Coordinates:
  * lat           (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) datetime64[ns] 152B 2016-09-01T12:00:00 ... 2016-09-...
Data variables:
    analysed_sst  (time, lat, lon) float64 158MB dask.array<chunksize=(10, 720, 720), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.4
    acknowledgment:             Funded by ESA
    cdm_data_type:              grid
    comment:                    
    creator_email:              science.leader@esa-sst-cci.org
    creator_name:               SST_cci
    ...                         ...
    summary:                    An ensemble product with input from a number ...
    time_coverage_end:          20170101T000000Z
    time_coverage_start:        20161231T000000Z
    title:                      Global SST Ensemble, L4 GMPE
    uuid:                       dc0c5b25-93bf-4943-aba1-7f0de9109620
    westernmost_longitude:      -180.0

In [22]:
cci_dataset.attrs["title"] = "Global SST Ensemble, L4 GMPE 2016"

Let's set some attributes of the variable, so the colormapping does not fall back on the default

In [23]:
cci_dataset.analysed_sst.attrs["color_value_min"] = 270
cci_dataset.analysed_sst.attrs["color_value_max"] = 310
cci_dataset.analysed_sst.attrs["color_bar_name"] = "plasma"

In [ ]:
viewer = Viewer()

In [25]:
viewer.add_dataset(dataset)
viewer.add_dataset(cci_dataset)

'2acfc9fb-9d09-4dcc-9c56-783536876220'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [26]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001
Viewer: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001


You can also open xcube Viewer inlined here:

In [27]:
viewer.show()

To stop the server and viewer:

In [28]:
viewer.stop_server()

---
**Scenario 3**: Use custom server configuration to start server and pass it to the viewer constructor. In this case, we have created a local file with the configuration and load it as a dictionary and pass it to the viewer. 

The custom configuration allows you to predefine your value ranges, the colormaps that should be used as well as which bands should be used to create an RGB image, then the RGB switch in the viewer will display the RGB image. 

If you do not have a server-config.yaml file in your directory, please create one with the following content: 

```yaml
DataStores:
  - Identifier: deep-esdl-cci-sst
    StoreId: s3
    StoreParams:
      root: $S3_USER_STORAGE_BUCKET
      storage_options:
          anon: false
          key: $S3_USER_STORAGE_KEY
          secret: $S3_USER_STORAGE_SECRET
    Datasets:
      - Path: "*.zarr"
        Style: default

        # ChunkCacheSize: 1G


Styles:
  - Identifier: default
    ColorMappings:
      analysed_sst:
        ColorBar: plasma
        ValueRange: [270, 310]
## if you have bands that can create an RGB image, you can specify them as below. 
#      rgb:
#        Red:
#          Variable: B04
#          ValueRange: [0., 0.25]
#        Green:
#          Variable: B03
#          ValueRange: [0., 0.25]
#        Blue:
#          Variable: B02
#          ValueRange: [0., 0.25]

```

**For all possible settings within the server configuration file, please checkout the documentation:** https://xcube.readthedocs.io/en/latest/cli/xcube_serve.html#configuration-file

In [29]:
from xcube.util.config import load_configs

In [ ]:
viewer = Viewer(server_config=load_configs("server-config.yaml"))

In [31]:
viewer.show()

To stop the server and viewer:

In [32]:
viewer.stop_server()

After all our testing and exploring, let's clean up the example cube :) 

In [33]:
team_store.delete_data(output_id)